# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
1.2927e19,"""11465922221560…","""12836691598823…","""../../../../da…","""13319416758030…","""59427797627023…",false
1.4856e19,"""12674646606013…","""10939424358753…","""../../../../da…","""13319416758030…","""12499658257858…",true
5.6458e17,"""91012588057315…","""36421672761923…","""../../../../da…","""35664037547327…","""59427797627023…",false
4.0723e18,"""12674646606013…","""10829150240662…","""../../../../da…","""62196553675314…","""59427797627023…",false
1.4677e19,"""11465922221560…","""84257753280170…","""../../../../da…","""62196553675314…","""47684181719641…",true
1.6330e19,"""91012588057315…","""45360284748457…","""../../../../da…","""17245759937656…",null,false
8.7675e18,"""91012588057315…","""35489803983272…","""../../../../da…","""62196553675314…","""47684181719641…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""62598315273423…","""13434883589077…","""../../../../da…","""FirstMortgage3…","""60953112443905…","""Rejected"""
1.4856e19,"""59453543710598…","""97012576451818…","""../../../../da…","""FirstMortgage3…","""13346135536676…","""Accepted"""
5.6458e17,"""17170012924505…","""65604523068591…","""../../../../da…","""MoneyMarketSav…","""60953112443905…","""Rejected"""
4.0723e18,"""59453543710598…","""14855878572080…","""../../../../da…","""BasicChecking""","""60953112443905…","""Rejected"""
1.4677e19,"""62598315273423…","""22378531333229…","""../../../../da…","""BasicChecking""","""12393908373202…","""Accepted"""
1.6330e19,"""17170012924505…","""11276078748849…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""17170012924505…","""15570640177302…","""../../../../da…","""BasicChecking""","""12393908373202…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""12294778322510…","""67144226894333…","""../../../../da…","""FirstMortgage3…","""14004924338447…","""Rejected"""
1.4856e19,"""14335060440940…","""40173693784510…","""../../../../da…","""FirstMortgage3…","""18437221179624…","""Accepted"""
5.6458e17,"""14811546033495…","""91627448763593…","""../../../../da…","""MoneyMarketSav…","""14004924338447…","""Rejected"""
4.0723e18,"""14335060440940…","""63250386186805…","""../../../../da…","""BasicChecking""","""14004924338447…","""Rejected"""
1.4677e19,"""12294778322510…","""10812012060135…","""../../../../da…","""BasicChecking""","""74835811446968…","""Accepted"""
1.6330e19,"""14811546033495…","""98362711541310…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""14811546033495…","""20817223718154…","""../../../../da…","""BasicChecking""","""74835811446968…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_MaritalStatus=PREDICTOR_1
Customer_City=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.2927e19,"""62897578553942…","""89029263118809…","""FirstMortgage3…","""15795905244509…","""Rejected"""
1.4856e19,"""43469133889789…","""11795728684309…","""FirstMortgage3…","""11776669556239…","""Accepted"""
5.6458e17,"""85956955466478…","""23237555322748…","""MoneyMarketSav…","""15795905244509…","""Rejected"""
4.0723e18,"""43469133889789…","""50953509685299…","""BasicChecking""","""15795905244509…","""Rejected"""
1.4677e19,"""62897578553942…","""85712974737455…","""BasicChecking""","""15521045855409…","""Accepted"""
1.6330e19,"""85956955466478…","""17839303481697…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""85956955466478…","""15333885845852…","""BasicChecking""","""15521045855409…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'